# EchoPro Python Workflow <a class="tocSkip">

# Import libraries and configure the Jupyter notebook

In [ ]:
# libraries used in the Notebook
import matplotlib.pyplot as plt

# Python version of EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.semivariogram import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
%matplotlib widget
# %matplotlib inline

# Set up  EchoPro for a specific survey year

## Initialize EchoPro object  using configuration files

* `initialization_config.yml` -- parameters independent of survey year
* `survey_year_2019_config.yml` -- parameters specific to survey year
* `source` -- Define the region of data to use e.g. US, CAN, US & CAN
* `exclude_age1` -- States whether age 1 hake should be included in analysis.

In [ ]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

## Load and process input data 
* This data is stored in `survey_2019`

In [ ]:
%%time 
survey_2019.load_survey_data() #file_type='biological')

In [ ]:
survey_2019.specimen_df.head()

## Compute the normalized biomass density
* The biomass density is stored in `survey_2019`

In [ ]:
%%time
survey_2019.compute_biomass_density()

In [ ]:
survey_2019.final_biomass_table.head()

# Jolly-Hampton CV Analysis

* Compute the mean of the Jolly-Hampton CV value on data that has not been Kriged
* Note: the algorithm used to compute this value is random in nature

In [ ]:
%%time
# CV_JH_mean = survey_2019.run_cv_analysis(kriged_data=False, seed=None)

In [ ]:
# print(f"Mean Jolly-Hampton CV: {CV_JH_mean:.4f}")

# Obtain Kriging Mesh Data

## Access Kriging mesh object
* Reads mesh data files specified by `survey_2019` 

In [ ]:
krig_mesh = survey_2019.get_kriging_mesh()

## Apply coordinate transformations
* Longitude transformation
* Lat/Lon to distance

### Transect points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='transect')

### Mesh points

In [ ]:
krig_mesh.apply_coordinate_transformation(coord_type='mesh')

# Perform Ordinary Kriging of biomass density

* transformed mesh points
* semi-variogram model
* normalized biomass density 

## Initialize Kriging routine

In [ ]:
kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel
)

# uncomment to use widget values 
# kriging_params.update(semi_vario.get_params_for_kriging())

# initalize kriging routine
krig = survey_2019.get_kriging(kriging_params)

## Perform Kriging
* Also generates total biomass at mesh points

In [ ]:
%%time
krig.run_biomass_kriging(krig_mesh)

In [ ]:
print(f"Total Kriged Biomass Estimate: {1e-6 * survey_2019.krig_results_gdf.krig_biomass_vals.sum():.3f} kmt")

In [ ]:
from EchoPro.cv_analysis.cv_analysis import get_transect_strata_info_no_kriging, get_transect_strata_info_kriged

In [ ]:
import numpy as np 
lat_inpfc = (np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000)

In [ ]:
survey_2019.krig_results_gdf.head()

In [ ]:
# get_transect_strata_info_no_kriging

In [ ]:
%%time
CV_JH_mean_kriged = survey_2019.run_cv_analysis(kriged_data=True, seed=None)
# dist from mat
# start_end = [[0, 7], [7, 30], [30, 42], [42, 56], [56, 70], [70, 103]]

In [ ]:
print(f"Kriged Mean Jolly-Hampton CV: {CV_JH_mean_kriged:.4f}")

In [ ]:
# transect_info, strata_info = get_transect_strata_info_no_kriging(lat_inpfc,
#                                                                          survey_2019.final_biomass_table)

# transect_info, strata_info = get_transect_strata_info_kriged(lat_inpfc,
#                                                              survey_2019.krig_results_gdf)